<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.63
    Uninstalling yfinance-0.2.63:
      Successfully uninstalled yfinance-0.2.63
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-16 11:46:38
-------------------
qualified stocks: 92
with latest results: 29
still star stocks: 17
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  1.50 L
Current:  1.33 C
-------------------
Today PnL: -27.09 K (-0.2%)
Current PnL: -16.62 L (-11.58%)
CY Booked + Current PnL: -10.25 L (-7.14%)
-------------------
Total profit:  3.02 L
Total loss:  -19.64 L
-------------------
Total Booked + Current PnL: 16.73 L (14.36%)
Total Booked PnL: 33.35 L (28.62%)
Curr Year Booked PnL: 6.37 L (4.8%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.05 C
Est FTT PnL: 72.73 L (54.86%)
Deployed:  1.17 C
Current:  1.33 C
CAGR/XIRR %: 8.99%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LICI,827.06,983.0,-5.02,H-LC,89.13,96928.0,11741.0,4323.0,0.33,13.78,4.46,18.85,35.0,2.72,0.74,29.05,XY25,NTT,INSURANCE
61,RELIANCE,1291.83,1526.0,5.74,X-LC,34.78,156862.0,16053.0,9474.0,0.78,11.40,6.04,18.13,17.0,1.69,1.20,23.86,XY25,NTT,REFINERIES
3,ADANIPORTS,1103.69,1583.0,-0.76,M-LC,43.48,99535.0,21173.0,12860.0,-0.22,27.02,12.92,43.43,77.0,1.65,0.76,33.78,XY24,NTT,MISC
77,TATAELXSI,7332.28,7332.0,-15.14,X-MC,64.13,89040.0,-13612.0,13605.0,-0.51,-13.26,15.28,-0.00,57.0,-1.00,0.68,35.35,OX40N,NTT,IT
88,VIPIND,488.80,489.0,-956.87,H-SC,88.04,81617.0,-13699.0,13736.0,3.46,-14.37,16.83,0.04,153.0,-1.00,0.62,60.98,OX40N,NTT,MISC


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,VOLTAS,1278.28,1929.20,-29.20,H-MC,53.26,192855.0,1113.0,96524.0,0.67,0.58,50.05,50.92,99.0,0.01,1.47,6.82,XY25,ATH,AC
10,BAJAJHFL,122.26,152.00,-7.52,H-LC,28.26,199014.0,-1492.0,50271.0,-0.23,-0.74,25.26,24.33,78.0,-0.03,1.52,14.76,X40N,BTT,FINANCE
57,PGHH,13388.00,18106.42,-20.27,X-MC,9.78,201150.0,330.0,70443.0,-0.27,0.16,35.02,35.24,53.0,0.00,1.53,4.85,X40,ATH,FMCG
34,HINDALCO,651.95,761.55,-13.37,H-LC,30.43,103456.0,-856.0,18394.0,0.75,-0.82,17.78,16.81,26.0,-0.05,0.79,15.09,X5K,ATH,METALS
36,HINDZINC,514.80,744.74,36.63,H-LC,95.65,102210.0,-750.0,46741.0,-0.62,-0.73,45.73,44.67,25.0,-0.02,0.78,31.90,X5K,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,VIPIND,488.80,489.0,-956.87,H-SC,88.04,81617.0,-13699.0,13736.0,3.46,-14.37,16.83,0.04,153.0,-1.00,0.62,60.98,OX40N,NTT,MISC
68,SFL,1016.35,1287.0,291.65,H-SC,71.20,176936.0,-85282.0,155102.0,2.79,-32.52,87.66,26.63,145.0,-0.55,1.35,10.01,XY24,NTT,MISC
79,TCS,3794.03,4998.0,-16.16,X-LC,39.13,272992.0,-22942.0,116841.0,1.57,-7.75,42.80,31.73,2.0,-0.20,2.08,9.27,X40,BTT,IT
20,CIPLA,1495.00,1795.0,-16.30,H-LC,54.35,102416.0,2251.0,17851.0,1.55,2.25,17.43,20.07,29.0,0.13,0.78,10.39,X40N,BTT,PHARMA
53,MASFIN,326.60,399.5,-22.31,H-SC,81.52,88470.0,-9510.0,31380.0,1.55,-9.71,35.47,22.32,141.0,-0.30,0.67,28.31,XR,ATH,FINANCE


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TATAMOTORS,761.77,1065.00,-55.10,M-LC,13.04,248038.0,-28485.0,138554.0,-4.04,-10.30,55.86,39.81,69.0,-0.21,1.89,18.87,XY24,NTT,AUTO
85,VAIBHAVGBL,350.21,521.00,58.13,X-SC,22.83,105462.0,-46529.0,120649.0,-3.36,-30.61,114.40,48.77,61.0,-0.39,0.80,21.14,XR,NTT,JEWELLERY
83,UJJIVANSFB,52.77,53.00,48.19,M-SC,80.43,128709.0,-13770.0,14390.0,-2.85,-9.66,11.18,0.44,242.0,-0.96,0.98,51.91,OX40N,NTT,BANKS
22,COFFEEDAY,59.14,80.00,-55.73,L-SC,97.83,66989.0,-46560.0,86610.0,-2.70,-41.00,129.29,35.27,260.0,-0.54,0.51,63.96,XR,NTT,HOTELS
76,TANLA,917.30,1963.11,-30.98,H-SC,94.57,169415.0,-70000.0,342964.0,-2.63,-29.24,202.44,114.01,128.0,-0.20,1.29,55.38,AR,ATH,IT


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,UJJIVANSFB,52.77,53.0,48.19,M-SC,80.43,128709.0,-13770.0,14390.0,-2.85,-9.66,11.18,0.44,242.0,-0.96,0.98,51.91,OX40N,NTT,BANKS
75,SYMPHONY,1306.42,1306.0,-30.32,M-SC,2.17,148593.0,-22548.0,22497.0,-0.34,-13.18,15.14,-0.03,190.0,-1.00,1.13,6.80,OX40N,NTT,DURABLES
77,TATAELXSI,7332.28,7332.0,-15.14,X-MC,64.13,89040.0,-13612.0,13605.0,-0.51,-13.26,15.28,-0.00,57.0,-1.00,0.68,35.35,OX40N,NTT,IT
88,VIPIND,488.80,489.0,-956.87,H-SC,88.04,81617.0,-13699.0,13736.0,3.46,-14.37,16.83,0.04,153.0,-1.00,0.62,60.98,OX40N,NTT,MISC
19,CERA,8421.60,8422.0,-12.17,X-SC,79.35,90584.0,-18897.0,18905.0,-0.43,-17.26,20.87,0.00,65.0,-1.00,0.69,37.10,OX40N,NTT,CERAMICS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,INFY,1461.46,2275.00,-9.55,X-LC,61.96,289622.0,28021.0,117615.0,1.00,10.71,40.61,55.67,1.0,0.24,2.21,17.47,X40,BTT,IT
79,TCS,3794.03,4998.00,-16.16,X-LC,39.13,272992.0,-22942.0,116841.0,1.57,-7.75,42.80,31.73,2.0,-0.20,2.08,9.27,X40,BTT,IT
35,HINDUNILVR,2413.81,2723.00,-16.95,X-LC,27.17,261016.0,-9331.0,43955.0,0.50,-3.45,16.84,12.81,5.0,-0.21,1.99,8.03,XY25,NTT,FMCG
15,BRITANNIA,4983.33,6446.05,11.41,X-LC,48.91,255369.0,26136.0,41140.0,-0.33,11.40,16.11,29.35,7.0,0.64,1.95,21.34,XY25,ATH,FMCG
56,NESTLEIND,2268.60,2755.00,-12.09,X-LC,38.04,279104.0,13678.0,43233.0,0.37,5.15,15.49,21.44,10.0,0.32,2.13,11.67,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ADANIPORTS,1103.69,1583.0,-0.76,M-LC,43.48,99535.0,21173.0,12860.0,-0.22,27.02,12.92,43.43,77.0,1.65,0.76,33.78,XY24,NTT,MISC


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,4311.81,6039.03,0.48,H-MC,47.83,153531.0,2618.0,57835.0,1.22,1.73,37.67,40.06,100.0,0.05,1.17,25.58,AR,ATH,AUTO
3,ADANIPORTS,1103.69,1583.00,-0.76,M-LC,43.48,99535.0,21173.0,12860.0,-0.22,27.02,12.92,43.43,77.0,1.65,0.76,33.78,XY24,NTT,MISC
18,CAMS,3643.00,5250.99,-1.11,H-SC,55.43,111278.0,9274.0,35754.0,-0.16,9.09,32.13,44.14,125.0,0.26,0.85,26.81,X40N,ATH,MISC
84,UNIONBANK,123.87,163.00,-4.91,M-LC,90.22,166684.0,25844.0,18652.0,0.25,18.35,11.19,31.59,86.0,1.39,1.27,44.79,XY24,NTT,BANKS
91,WIPRO,243.46,420.00,-5.63,M-LC,58.70,161963.0,11018.0,98441.0,0.39,7.30,60.78,72.51,68.0,0.11,1.24,11.57,XR,NTT,IT


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
90,WHIRLPOOL,1167.49,2270.00,-33.78,M-SC,50.00,192938.0,21317.0,140748.0,-1.22,12.42,72.95,94.43,194.0,0.15,1.47,42.40,XR,NTT,DURABLES
91,WIPRO,243.46,420.00,-5.63,M-LC,58.70,161963.0,11018.0,98441.0,0.39,7.30,60.78,72.51,68.0,0.11,1.24,11.57,XR,NTT,IT
40,INDIAMART,2327.09,4850.92,-49.23,H-SC,82.61,132171.0,8835.0,124928.0,1.02,7.16,94.52,108.45,122.0,0.07,1.01,31.12,AR,ATH,MISC
33,HEROMOTOCO,4311.81,6039.03,0.48,H-MC,47.83,153531.0,2618.0,57835.0,1.22,1.73,37.67,40.06,100.0,0.05,1.17,25.58,AR,ATH,AUTO
12,BANKINDIA,116.91,190.00,-30.66,M-MC,76.09,185298.0,5490.0,106917.0,-0.59,3.05,57.70,62.52,170.0,0.05,1.41,33.35,XR,NTT,BANKS


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.0,-55.73,L-SC,97.83,66989.0,-46560.0,86610.0,-2.70,-41.00,129.29,35.27,260.0,-0.54,0.51,63.96,XR,NTT,HOTELS
7,ASIANTILES,75.41,137.0,7422.22,M-SC,100.00,82527.0,-11283.0,87900.0,-0.42,-12.03,106.51,81.67,238.0,-0.13,0.63,59.82,XR,NTT,CERAMICS
53,MASFIN,326.60,399.5,-22.31,H-SC,81.52,88470.0,-9510.0,31380.0,1.55,-9.71,35.47,22.32,141.0,-0.30,0.67,28.31,XR,ATH,FINANCE
51,LICI,827.06,983.0,-5.02,H-LC,89.13,96928.0,11741.0,4323.0,0.33,13.78,4.46,18.85,35.0,2.72,0.74,29.05,XY25,NTT,INSURANCE
3,ADANIPORTS,1103.69,1583.0,-0.76,M-LC,43.48,99535.0,21173.0,12860.0,-0.22,27.02,12.92,43.43,77.0,1.65,0.76,33.78,XY24,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,75.41,137.00,7422.22,M-SC,100.00,82527.0,-11283.0,87900.0,-0.42,-12.03,106.51,81.67,238.0,-0.13,0.63,59.82,XR,NTT,CERAMICS
86,VALIANTORG,512.64,838.00,-378.23,H-SC,98.91,118404.0,-14882.0,99471.0,-2.59,-11.17,84.01,63.47,142.0,-0.15,0.90,98.47,XR,NTT,CHEMICALS
22,COFFEEDAY,59.14,80.00,-55.73,L-SC,97.83,66989.0,-46560.0,86610.0,-2.70,-41.00,129.29,35.27,260.0,-0.54,0.51,63.96,XR,NTT,HOTELS
47,JPPOWER,18.73,26.20,-34.40,L-SC,96.74,132982.0,-8860.0,65427.0,-0.62,-6.25,49.20,39.88,258.0,-0.14,1.01,34.77,XY24,NTT,POWER
36,HINDZINC,514.80,744.74,36.63,H-LC,95.65,102210.0,-750.0,46741.0,-0.62,-0.73,45.73,44.67,25.0,-0.02,0.78,31.90,X5K,ATH,METALS


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.58
1,25,42.88
2,50,71.80


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.73
LC    37.57
MC    21.98
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.51
XY25     14.19
X40      12.56
XR       10.43
X40N      9.90
OX40N     8.39
AR        8.03
X5K       2.32
X200      1.50
SR        1.17
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    19.41
X-LC    15.54
M-SC    14.15
H-LC    12.59
H-MC     8.60
M-LC     8.33
X-MC     7.66
M-MC     5.72
X-SC     2.75
L-SC     2.42
L-LC     1.11
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.50,-3.00,37.46
IT,11.36,-11.58,68.26
BANKS,8.03,-8.38,52.61
FINANCE,7.86,-17.16,59.49
MISC,6.57,-25.49,74.44
PAINTS,5.58,-19.19,42.68
ELECTRICAL,5.14,-4.34,49.53
HEALTHCARE,4.76,-3.45,33.67
AUTO,4.56,-12.92,62.57


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2800393.0,25
XR,1089369.0,13
AR,1017043.0,9
X40,672475.0,9
XY25,530436.0,10
X40N,481674.0,9
OX40N,331077.0,11
SR,190488.0,2
X5K,97547.0,3


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1952081.0,19
M-SC,1472673.0,18
X-LC,626563.0,9
M-MC,594151.0,5
H-LC,556472.0,11
H-MC,497944.0,7
X-MC,412661.0,6
M-LC,352678.0,6
L-SC,278958.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,912138.0,6
M-SC,XY24,589550.0,5
H-SC,AR,523952.0,3
M-MC,XY24,399200.0,3
M-SC,XR,347739.0,4
X-LC,X40,315784.0,3
M-LC,XY24,234989.0,4
H-MC,XY24,227806.0,2
H-SC,XR,208295.0,3


In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
